In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("Yale-LILY/brio-cnndm-uncased")
model = AutoModelForSeq2SeqLM.from_pretrained("Yale-LILY/brio-cnndm-uncased").to(device)

In [2]:
device

'cuda'

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/teamspace/studios/this_studio/NLP_project/openai_cnndm_final.csv', sep=';')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6291 entries, 0 to 6290
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.1  6291 non-null   int64 
 1   Unnamed: 0    6291 non-null   int64 
 2   user_id       6291 non-null   object
 3   user_profile  6291 non-null   object
 4   doc_id        6291 non-null   object
 5   post/article  6291 non-null   object
 6   summary_text  6291 non-null   object
 7   confidence    6291 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 393.3+ KB


In [6]:
tokenizer, model

(BartTokenizerFast(name_or_path='Yale-LILY/brio-cnndm-uncased', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
 	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
 },
 BartForConditionalGeneration(
   (model): BartModel(
     (shared): Embedding(502

In [7]:
data = df[['doc_id', 'post/article']]

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6291 entries, 0 to 6290
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   doc_id        6291 non-null   object
 1   post/article  6291 non-null   object
dtypes: object(2)
memory usage: 98.4+ KB


In [9]:
mapp = {}

for idx , row in data.iterrows():
    doc_id = row['doc_id']
    text = row['post/article']

    if doc_id not in mapp:
        mapp[doc_id] = text

In [10]:
len(mapp)

639

In [11]:
ids = list(mapp.keys())
len(ids)

639

In [12]:
unique_texts = list(mapp.values())
len(unique_texts)

639

In [17]:
batch_size = 5

def translate_batch(texts):
    inputs = tokenizer(texts, max_length=1024, truncation=True, padding="max_length", return_tensors="pt").to(device)
    summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=20)

    summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return summaries

In [26]:
results = []

In [27]:
src_texts = unique_texts
len(src_texts)

639

In [28]:
for i in range(0, len(src_texts), batch_size):
    batch_texts = src_texts[i:i + batch_size]
    translated_texts = translate_batch(batch_texts)
    results.extend(translated_texts)


In [29]:
len(results)

639

In [30]:
gen_summaries = results

In [31]:
mapp = dict(zip(ids, gen_summaries))

In [32]:
len(mapp)

639

In [33]:
df['brio_model_summary'] = df['doc_id'].map(mapp)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6291 entries, 0 to 6290
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0.1        6291 non-null   int64 
 1   Unnamed: 0          6291 non-null   int64 
 2   user_id             6291 non-null   object
 3   user_profile        6291 non-null   object
 4   doc_id              6291 non-null   object
 5   post/article        6291 non-null   object
 6   summary_text        6291 non-null   object
 7   confidence          6291 non-null   int64 
 8   brio_model_summary  6291 non-null   object
dtypes: int64(3), object(6)
memory usage: 442.5+ KB


In [35]:
df.to_csv('openai_cnndm_final_brio.csv', sep=';')

In [36]:
data = pd.read_csv('openai_cnndm_final_brio.csv', sep=';')

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6291 entries, 0 to 6290
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0.2        6291 non-null   int64 
 1   Unnamed: 0.1        6291 non-null   int64 
 2   Unnamed: 0          6291 non-null   int64 
 3   user_id             6291 non-null   object
 4   user_profile        6291 non-null   object
 5   doc_id              6291 non-null   object
 6   post/article        6291 non-null   object
 7   summary_text        6291 non-null   object
 8   confidence          6291 non-null   int64 
 9   brio_model_summary  6291 non-null   object
dtypes: int64(4), object(6)
memory usage: 491.6+ KB


In [42]:
data.iloc[30]['brio_model_summary'], results[2]

('wooden deckchair was salvaged from the Atlantic Ocean after the Titanic sank in 1912',
 'wooden deckchair was salvaged from the Atlantic Ocean after the Titanic sank in 1912')